In [ ]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC

from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [3]:
read_json = lambda x : pd.read_json(x, lines = True)

In [4]:
train_X = read_json("train_X_languages_homework.json.txt").text
train_y = read_json("train_y_languages_homework.json.txt").classification
test_X = read_json("test_X_languages_homework.json.txt").text
labels = train_y.unique().tolist()
pd_data = pd.concat([train_X, train_y], axis = 1)
print pd_data.size, pd_data.size
pd_data = pd_data.drop_duplicates()
train_X = pd_data.text.reset_index(drop=True)
train_y = pd_data.classification.reset_index(drop=True)

250782 250782


In [ ]:
train_X.iloc[122765]

In [ ]:
pd_data = pd.concat([train_X, train_y], axis = 1)
groupby_class = pd_data["text"].groupby(pd_data['classification'])

In [ ]:
desc = groupby_class.describe()

In [ ]:
desc[["count", "unique"]].plot.bar(figsize = (20, 20), fontsize = 20)

In [ ]:
dup_counts = (desc["count"] - 1.0 * desc["unique"])
frac_dup = dup_counts/ desc["count"]
frac_dup.plot.bar(figsize = (20, 20), fontsize = 20)
# Overall data duplication percentage
print dup_counts.sum()/ desc["count"].sum()

In [ ]:
avg_tokens_per_sent = groupby_class.agg({'text' : lambda x : sum(len(word_tokenize(y))*1.0/len(x) for y in x)})

In [ ]:
avg_tokens_per_sent.plot.bar(figsize = (20, 20), fontsize = 20)

In [5]:
class TextTokenizer(BaseEstimator, TransformerMixin):
    def fit(self, x, y = None):
        return self
    
    def transform(self, sentences):
        features = []
        tokenizer = RegexpTokenizer('\w+')
        for sentence in sentences:
            features.append(" ".join(tokenizer.tokenize(sentence)))
        return features

class TextStats(BaseEstimator, TransformerMixin):
    def fit(self, x, y = None):
        return self
    def transform(self, sent_tokens):
        return [{'num_words': sent_toks.count(' ') + 1 } 
                for sent_toks in sent_tokens]

In [25]:
from tempfile import mkdtemp
from shutil import rmtree
from sklearn.externals.joblib import Memory
cachedir = mkdtemp()
memory = Memory(cachedir=cachedir, verbose = 0)
kappa_scorer = make_scorer(cohen_kappa_score)
# detection_pipeline = Pipeline([
#     ('tokenizer', TextTokenizer()),
#     ('vect', TfidfVectorizer(ngram_range=(1,6), analyzer = 'char_wb')),
#     ('features', SelectKBest(score_func = chi2)),
#     ('clf', LogisticRegression(C = 21.5443469, 
#                                  solver = 'sag', 
#                                  multi_class = 'multinomial',
#                                  verbose = 1))
# ], memory = memory)

detection_pipeline = Pipeline([
    ('tokens', TextTokenizer()),
    
    ('features', FeatureUnion([
        ('vect', Pipeline([
            ('algo', CountVectorizer(ngram_range = (4, 6), analyzer='char_wb', token_pattern=r'\s*(.*?)\s',  max_features= 40000)),
            ('scale', Normalizer())
        ])),
        ('text_stats', Pipeline([
            ('stats', TextStats()),
            ('vect', DictVectorizer()),
            ('scale', Normalizer())
        ])),
    ])),
    ('clf', None),
], memory = memory)

In [ ]:
kappa_scorer = make_scorer(cohen_kappa_score)
clf = GridSearchCV(estimator=detection_pipeline,
                  param_grid = {
#                       'features__vect__algo)' : (
# #                                    CountVectorizer(ngram_range = (4, 5), analyzer='char_wb'),
#                                    CountVectorizer(ngram_range = (1, 6), analyzer='char_wb', token_pattern=r'\s*(.*?)\s'),
#                                ),
                      'clf' : (
                                   SGDClassifier(loss = 'log',
#                                       multi_class= 'multinomial',
                                      tol = 1e-4,
                                      max_iter = 100,
                                      verbose = 1),
                              ),
                      'clf__class_weight': ('balanced',),
#                       'clf__solver': ('sag',),
#                       'clf__C': (1.0,),
#                       'features__vect__algo__max_features': (40000,),
                  },
                  cv = 5,
                  scoring = kappa_scorer,
                  return_train_score = True,
                  n_jobs = -1)
kf = StratifiedKFold(n_splits=2, shuffle = True, random_state = 5)
all_clfs = []
all_predicts = []
all_test = []
num = 1
for train_index, test_index in kf.split(train_X, train_y):
    X_train, X_test = train_X[train_index], train_X[test_index]
    y_train, y_test = train_y[train_index], train_y[test_index]
    print X_train.hasnans, y_train.hasnans
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print "=========== %d ==========" % num
    print "Cohen Kappa:", cohen_kappa_score(y_pred, y_test)
    print "Accuracy:", accuracy_score(y_test, y_pred)
    print classification_report(y_test, y_pred)
    print clf.best_params_
    print "========================="
    all_predicts.append(y_pred)
    all_test.append(y_test)
    all_clfs.append(clf)
    num += 1
    break
rmtree(cachedir)

False False
-- Epoch 1
Norm: 16.65, NNZs: 40001, Bias: -0.203309, T: 49097, Avg. loss: 0.084594
Total training time: 0.06 seconds.
-- Epoch 2
Norm: 16.62, NNZs: 40001, Bias: -0.231641, T: 98194, Avg. loss: 0.080215
-- Epoch 1
Total training time: 0.15 seconds.
-- Epoch 3
Norm: 16.71, NNZs: 40001, Bias: -0.201585, T: 49096, Avg. loss: 0.085473
Total training time: 0.04 seconds.
-- Epoch 2
Norm: 16.63, NNZs: 40001, Bias: -0.246690, T: 147291, Avg. loss: 0.080101
Total training time: 0.23 seconds.
-- Epoch 4
Norm: 16.69, NNZs: 40001, Bias: -0.228583, T: 98192, Avg. loss: 0.081503
Total training time: 0.12 seconds.
Norm: 16.62, NNZs: 40001, Bias: -0.256337, T: 196388, Avg. loss: 0.080172
-- Epoch 3
Total training time: 0.28 seconds.
Convergence after 4 epochs took 0.30 seconds
-- Epoch 1
Norm: 16.62, NNZs: 40001, Bias: -0.242740, T: 147288, Avg. loss: 0.081255
Total training time: 0.20 seconds.
Norm: 16.41, NNZs: 40001, Bias: -0.215766, T: 49097, Avg. loss: 0.057761
Total training time: 0.

Total training time: 0.37 seconds.
Convergence after 6 epochs took 0.37 seconds
-- Epoch 1
Norm: 16.13, NNZs: 40001, Bias: -0.211363, T: 49097, Avg. loss: 0.069349
Total training time: 0.05 seconds.
Norm: 18.45, NNZs: 40001, Bias: -0.303960, T: 98194, Avg. loss: 0.016330
Total training time: 0.13 seconds.
-- Epoch 2
-- Epoch 3
Norm: 17.78, NNZs: 40001, Bias: -0.252658, T: 49096, Avg. loss: 0.037225
Total training time: 0.06 seconds.
Norm: 15.95, NNZs: 40001, Bias: -0.237721, T: 98194, Avg. loss: 0.064285
Total training time: 0.10 seconds.
-- Epoch 2
Norm: 18.65, NNZs: 40001, Bias: -0.331834, T: 147291, Avg. loss: 0.016552
-- Epoch 3
Total training time: 0.18 seconds.
Convergence after 3 epochs took 0.18 seconds
-- Epoch 1
Norm: 16.92, NNZs: 40001, Bias: -0.276230, T: 98192, Avg. loss: 0.024586
Total training time: 0.12 seconds.
Norm: 15.93, NNZs: 40001, Bias: -0.252840, T: 147291, Avg. loss: 0.063943
Total training time: 0.16 seconds.
-- Epoch 4
Norm: 15.09, NNZs: 40001, Bias: -0.21527

Norm: 14.88, NNZs: 40001, Bias: -0.240410, T: 98194, Avg. loss: 0.066579
Total training time: 0.07 seconds.
-- Epoch 3
Total training time: 0.12 seconds.
-- Epoch 3
Norm: 14.09, NNZs: 40001, Bias: -0.212721, T: 49096, Avg. loss: 0.088965
Total training time: 0.08 seconds.
-- Epoch 2
Norm: 16.83, NNZs: 40001, Bias: -0.273802, T: 245485, Avg. loss: 0.053666
Total training time: 0.22 seconds.
Convergence after 5 epochs took 0.22 seconds
Norm: 15.89, NNZs: 40001, Bias: -0.255835, T: 147291, Avg. loss: 0.065360
Total training time: 0.11 seconds.
-- Epoch 1
-- Epoch 4
Norm: 14.89, NNZs: 40001, Bias: -0.256529, T: 147291, Avg. loss: 0.066385
Total training time: 0.17 seconds.
-- Epoch 4
Norm: 15.85, NNZs: 40001, Bias: -0.265593, T: 196388, Avg. loss: 0.065097
Norm: 13.96, NNZs: 40001, Bias: -0.237953, T: 98192, Avg. loss: 0.083055
Norm: 13.16, NNZs: 40001, Bias: -0.213422, T: 49097, Avg. loss: 0.114475
Total training time: 0.05 seconds.
Total training time: 0.16 seconds.
-- Epoch 2
Total trai

-- Epoch 5
-- Epoch 5
Total training time: 0.25 seconds.
-- Epoch 4
Norm: 17.01, NNZs: 40001, Bias: -0.257419, T: 147291, Avg. loss: 0.032667
Total training time: 0.33 seconds.
-- Epoch 4
Norm: 16.66, NNZs: 40001, Bias: -0.274381, T: 245485, Avg. loss: 0.055259
Total training time: 0.32 seconds.
Norm: 18.23, NNZs: 40001, Bias: -0.353356, T: 245485, Avg. loss: 0.016155
Total training time: 0.47 seconds.
-- Epoch 6
Convergence after 5 epochs took 0.47 seconds
Norm: 13.18, NNZs: 40001, Bias: -0.268327, T: 196384, Avg. loss: 0.109843
Total training time: 0.36 seconds.
Convergence after 4 epochs took 0.36 seconds
-- Epoch 1
Norm: 17.03, NNZs: 40001, Bias: -0.272130, T: 196388, Avg. loss: 0.032581
Total training time: 0.41 seconds.
-- Epoch 1
Convergence after 4 epochs took 0.41 seconds
-- Epoch 1
Norm: 16.62, NNZs: 40001, Bias: -0.280100, T: 294582, Avg. loss: 0.055315
Total training time: 0.39 seconds.
Norm: 15.00, NNZs: 40001, Bias: -0.221813, T: 49096, Avg. loss: 0.059343
Convergence aft

-- Epoch 1
-- Epoch 2
Convergence after 5 epochs took 0.27 seconds
-- Epoch 1
Norm: 14.49, NNZs: 40001, Bias: -0.253954, T: 147291, Avg. loss: 0.121852
Total training time: 0.20 seconds.
-- Epoch 4
Norm: 16.98, NNZs: 40001, Bias: -0.210637, T: 49097, Avg. loss: 0.059806
Norm: 17.31, NNZs: 40001, Bias: -0.221216, T: 49096, Avg. loss: 0.050001
Total training time: 0.05 seconds.
Norm: 15.22, NNZs: 40001, Bias: -0.236935, T: 98194, Avg. loss: 0.096532
Total training time: 0.04 seconds.
Total training time: 0.11 seconds.
-- Epoch 2
-- Epoch 3
-- Epoch 2
Norm: 14.47, NNZs: 40001, Bias: -0.264570, T: 196388, Avg. loss: 0.121909
Total training time: 0.24 seconds.
Convergence after 4 epochs took 0.25 seconds
Norm: 17.13, NNZs: 40001, Bias: -0.247655, T: 98192, Avg. loss: 0.044929
-- Epoch 1
Total training time: 0.08 seconds.
-- Epoch 3
Norm: 16.70, NNZs: 40001, Bias: -0.236249, T: 98194, Avg. loss: 0.055704
Total training time: 0.10 seconds.
-- Epoch 3
Norm: 15.21, NNZs: 40001, Bias: -0.253040,

...

-- Epoch 5
Norm: 15.90, NNZs: 40001, Bias: -0.273463, T: 245485, Avg. loss: 0.064247
Total training time: 0.14 seconds.
Convergence after 5 epochs took 0.14 seconds
-- Epoch 1
Norm: 14.22, NNZs: 40001, Bias: -0.221180, T: 49097, Avg. loss: 0.088308
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 14.14, NNZs: 40001, Bias: -0.246897, T: 98194, Avg. loss: 0.082784
Total training time: 0.05 seconds.
-- Epoch 3
Norm: 14.11, NNZs: 40001, Bias: -0.263014, T: 147291, Avg. loss: 0.082012
Total training time: 0.07 seconds.
-- Epoch 4
Norm: 14.05, NNZs: 40001, Bias: -0.273374, T: 196388, Avg. loss: 0.081668
Total training time: 0.10 seconds.
-- Epoch 5
Norm: 14.10, NNZs: 40001, Bias: -0.283586, T: 245485, Avg. loss: 0.081694
Total training time: 0.12 seconds.
Convergence after 5 epochs took 0.12 seconds
-- Epoch 1
Norm: 17.02, NNZs: 40001, Bias: -0.245163, T: 49097, Avg. loss: 0.035533
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 16.56, NNZs: 40001, Bias: -0.269391, T: 98194, Avg. loss

Total training time: 0.05 seconds.
-- Epoch 3
Norm: 17.05, NNZs: 40001, Bias: -0.256725, T: 147291, Avg. loss: 0.043688
Total training time: 0.08 seconds.
-- Epoch 4
Norm: 17.07, NNZs: 40001, Bias: -0.270805, T: 196388, Avg. loss: 0.043630
Total training time: 0.10 seconds.
Convergence after 4 epochs took 0.11 seconds
-- Epoch 1
Norm: 15.51, NNZs: 40001, Bias: -0.213332, T: 49097, Avg. loss: 0.102804
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 15.24, NNZs: 40001, Bias: -0.235146, T: 98194, Avg. loss: 0.096973
Total training time: 0.05 seconds.
-- Epoch 3
Norm: 15.20, NNZs: 40001, Bias: -0.250715, T: 147291, Avg. loss: 0.096549
Total training time: 0.08 seconds.
-- Epoch 4
Norm: 15.19, NNZs: 40001, Bias: -0.262227, T: 196388, Avg. loss: 0.096337
Total training time: 0.10 seconds.
-- Epoch 5
Norm: 15.23, NNZs: 40001, Bias: -0.272824, T: 245485, Avg. loss: 0.096128
Total training time: 0.12 seconds.
-- Epoch 6
Norm: 15.19, NNZs: 40001, Bias: -0.278400, T: 294582, Avg. loss: 0.0962

Total training time: 0.11 seconds.
Convergence after 4 epochs took 0.11 seconds
-- Epoch 1
Norm: 15.62, NNZs: 40001, Bias: -0.229619, T: 49097, Avg. loss: 0.049475
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 15.41, NNZs: 40001, Bias: -0.251304, T: 98194, Avg. loss: 0.044092
Total training time: 0.06 seconds.
-- Epoch 3
Norm: 15.26, NNZs: 40001, Bias: -0.265428, T: 147291, Avg. loss: 0.043250
Total training time: 0.08 seconds.
-- Epoch 4
Norm: 15.28, NNZs: 40001, Bias: -0.279234, T: 196388, Avg. loss: 0.043230
Total training time: 0.11 seconds.
Convergence after 4 epochs took 0.11 seconds
-- Epoch 1
Norm: 18.78, NNZs: 40001, Bias: -0.219013, T: 49097, Avg. loss: 0.051242
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 18.66, NNZs: 40001, Bias: -0.245819, T: 98194, Avg. loss: 0.047561
Total training time: 0.05 seconds.
-- Epoch 3
Norm: 18.60, NNZs: 40001, Bias: -0.260769, T: 147291, Avg. loss: 0.047357
Total training time: 0.08 seconds.
-- Epoch 4
Norm: 18.60, NNZs: 40001, Bi

Total training time: 0.21 seconds.
-- Epoch 6
Norm: 15.87, NNZs: 40001, Bias: -0.296019, T: 294582, Avg. loss: 0.038181
Total training time: 0.24 seconds.
Convergence after 6 epochs took 0.24 seconds
-- Epoch 1
Norm: 15.57, NNZs: 40001, Bias: -0.219365, T: 49097, Avg. loss: 0.055692
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 15.16, NNZs: 40001, Bias: -0.239998, T: 98194, Avg. loss: 0.049598
Total training time: 0.06 seconds.
-- Epoch 3
Norm: 15.11, NNZs: 40001, Bias: -0.257982, T: 147291, Avg. loss: 0.049113
Total training time: 0.10 seconds.
-- Epoch 4
Norm: 15.17, NNZs: 40001, Bias: -0.273185, T: 196388, Avg. loss: 0.048933
Total training time: 0.14 seconds.
-- Epoch 5
Norm: 15.11, NNZs: 40001, Bias: -0.279781, T: 245485, Avg. loss: 0.048950
Total training time: 0.18 seconds.
Convergence after 5 epochs took 0.19 seconds
-- Epoch 1
Norm: 16.21, NNZs: 40001, Bias: -0.208725, T: 49097, Avg. loss: 0.056390
Total training time: 0.04 seconds.
-- Epoch 2
Norm: 16.20, NNZs: 40001, B

Total training time: 0.08 seconds.
-- Epoch 4
Norm: 7.86, NNZs: 40001, Bias: -0.256498, T: 196388, Avg. loss: 0.068544
Total training time: 0.10 seconds.
-- Epoch 5
Norm: 7.81, NNZs: 40001, Bias: -0.263385, T: 245485, Avg. loss: 0.068460
Total training time: 0.13 seconds.
Convergence after 5 epochs took 0.13 seconds


In [ ]:
clf.cv_results_

CountVectorizer, MultinominalNb, (4,5) , char_wb


=========== 1 ==========

Cohen Kappa: 0.753482789861

Accuracy: 0.759855201888


             precision    recall  f1-score   support

         ar       0.91      0.93      0.92      2072
         az       0.93      0.85      0.88       960
         be       0.98      0.83      0.90      1008
         bg       0.90      0.85      0.87      1387
         ca       0.88      0.70      0.78      1507
         ce       0.99      0.42      0.59       288
        ceb       1.00      0.23      0.37       154
         cs       0.87      0.82      0.84      1166
         da       0.65      0.77      0.70      1491
         de       0.65      0.92      0.76      2349
         el       0.96      0.88      0.92      1383
         en       0.42      0.92      0.58      2266
         eo       0.86      0.76      0.81      1005
         es       0.42      0.92      0.58      2599
         et       0.98      0.81      0.88       462
         eu       0.95      0.84      0.89       887
         fa       0.96      0.83      0.89       992
         fi       0.96      0.84      0.90       799
         fr       0.67      0.86      0.75      1990
         gl       0.84      0.32      0.46      1071
         he       1.00      0.91      0.96      1153
         hi       0.99      0.82      0.90      1073
         hr       0.73      0.11      0.19       934
         hu       0.97      0.88      0.92      1367
         hy       0.99      0.88      0.94      1132
         id       0.57      0.77      0.66       838
         it       0.66      0.90      0.76      2418
         ja       0.97      0.46      0.62       864
         ka       0.99      0.92      0.96       584
         kk       0.97      0.75      0.85       560
         ko       1.00      0.30      0.46       617
         la       0.90      0.42      0.57       628
      lorem       0.98      1.00      0.99      1683
         lt       0.99      0.81      0.89       785
         ms       0.86      0.18      0.30       619
         nl       0.93      0.81      0.86      1233
         nn       0.86      0.51      0.64       697
         no       0.74      0.50      0.60      1192
         pl       0.85      0.80      0.82      1027
         pt       0.81      0.61      0.70      1356
         ro       0.85      0.78      0.81      1507
         ru       0.73      0.89      0.80      2023
         sh       0.47      0.82      0.60      1326
         sk       0.99      0.47      0.64       540
         sl       0.94      0.36      0.52       785
         sr       0.96      0.78      0.86      1014
         sv       0.87      0.70      0.78      1300
         th       1.00      0.33      0.49       789
         tr       0.91      0.80      0.85      1255
         uk       0.90      0.85      0.87      1715
         ur       1.00      0.79      0.88       457
         uz       0.98      0.74      0.84       445
         vi       0.88      0.87      0.87      1631
         vo       0.97      0.29      0.45       206
        war       0.85      0.50      0.63       274
         zh       1.00      0.04      0.09       845

avg / total       0.83      0.76      0.75     62708

